In [2]:
import sys
# print(sys.path)
sys.path.append('/home/sz/pa2023/')
print(sys.path)

['/home/sz/pa2023/test', '/usr/local/lib/python310.zip', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '', '/home/sz/pa2023/venv/lib/python3.10/site-packages', '/home/sz/pa2023/']


In [ ]:
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from dotenv import load_dotenv
load_dotenv()
from module.tools import tools_faiss_azure_googleserp
_tools = tools_faiss_azure_googleserp
_tools.append(WriteFileTool())
_tools.append(ReadFileTool())
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
_vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore   ({}), {})
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories import FileChatMessageHistory
_llm = ChatOpenAI(temperature=0)

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    _agent = AutoGPT.from_llm_and_tools(
        ai_name="Zack",
        ai_role="Assistant",
        tools=_tools,
        llm=_llm,
        memory=_vectorstore.as_retriever(),
        chat_history_memory=FileChatMessageHistory("chat_history.txt"),
    )
    _agent.chain.verbose = True
    _task = "Write a weather report for SF today"
    _re = _agent.run([_task])
    _token_cost = f"Tokens: {cb.total_tokens} = (Prompt {cb.prompt_tokens} + Completion {cb.completion_tokens}) Cost: ${format(cb.total_cost, '.5f')}"
    print(_token_cost)
    print(_re)